# SDP Product Equations

This notebook generates an overview of the formulas used to calculate product properties by the parametric model.

Last run with Jupyter Notebook 5.0.0 running Python 3.5.2

In [ ]:
import sys

from IPython.display import display, Math, Latex, HTML, Markdown
from sympy import latex, Lambda, Symbol

sys.path+=['..']
from sdp_par_model import evaluate
from sdp_par_model import reports as iapi
from sdp_par_model.config import PipelineConfig
from sdp_par_model.parameters.definitions import *
from sdp_par_model.parameters.container import BLDep

# Product properties to show
exprs = [('Rflop', 'PFLOP/s', Constants.peta),
         ('Rout', 'TB/s', Constants.tera)]

Python code to generate formulas:

In [ ]:
def show_formulas(sym_defs, group=1):
    maths = ""
    for i, sym in enumerate(sorted(sym_defs)):
        if isinstance(sym_defs[sym], BLDep):
            maths += r'%s(%s) &= %s %s' % (
                latex(Symbol(sym)),
                ",".join(sym_defs[sym].pars.keys()),
                latex(sym_defs[sym].term),
                r"\\" if i % group == 0 else " & ")
        else:
            maths += r'%s &= %s %s' % (latex(Symbol(sym)), latex(sym_defs[sym]),
                                      r"\\" if i % group == group-1 else " & ")
    display(Math(r"\begin{aligned}%s\end{aligned}" % maths))

def formulas_for_configuration(telescope, band, pipeline):
    cfg = PipelineConfig(telescope=telescope,band=band,pipeline=pipeline)
    if not cfg.is_valid()[0]:
        return

    # Heading
    display(Markdown("## Equations for %s %s:" % (telescope, pipeline)))

    # Loop through configuration to show formulas for
    free_syms = set()
    for expr, unit, mult  in exprs:

        # Determine and show product formulas
        values = cfg.eval_expression_products(expr)
        formulas = cfg.eval_products_symbolic(expr)

        # Show equations for products
        maths = ""
        for product in sorted(values.keys()):
            maths += (r'%s_{%s} &= %s \\ &= %f\,\text{%s} \\ \\' %
                (expr, product,
                 latex(formulas.get(product, 0)),
                 values.get(product, 0) / mult, unit))
        maths = (r"\begin{aligned}%s %s &= %f\,\text{%s}\end{aligned}" % \
            (maths, expr, sum(values.values()) / mult, unit))
        display(Math(maths))
            
        # Collect free symbols
        free_syms = free_syms.union(evaluate.collect_free_symbols(formulas.values()))

    # Look up helper equations
    display(Markdown("### Intermediate equations"))
    sym_defs = cfg.eval_symbols(free_syms, symbolify='helper')
    show_formulas(sym_defs)
    free_syms2 = evaluate.collect_free_symbols(sym_defs.values()).union(free_syms).difference(sym_defs)
    sym_defs2 = cfg.eval_symbols(free_syms2, symbolify='all')
    show_formulas(sym_defs2)

    # And finally all remaining constants
    display(Markdown("### Constants"))
    free_syms3 = evaluate.collect_free_symbols(sym_defs2.values()).\
                 union(free_syms, free_syms2).difference(sym_defs, sym_defs2)
    const_defs = cfg.eval_symbols(free_syms3, optimize_expression='Rflop')
    show_formulas(const_defs, 3)    

In [ ]:
for telescope, band in [(Telescopes.SKA1_Low, Bands.Low),
                        (Telescopes.SKA1_Mid, Bands.Mid1)]:
    for pipeline in [Pipelines.Ingest, Pipelines.ICAL,Pipelines.DPrepA_Image]:
        formulas_for_configuration(telescope, band, pipeline)